In [ ]:
import pandas as pd
import numpy as np

# from sentence_transformers import SentenceTransformer

In [ ]:
meta = pd.read_json('../data/raw/meta_Beauty.json.gz')

In [6]:
data = pd.read_csv('../data/preprocessed/Beauty.csv')
raw_data = pd.read_csv('../data/raw/Beauty.csv')
meta = pd.read_parquet('../meta_Beauty.parquet')

In [ ]:
sorted_ids = data.drop_duplicates('item_id').sort_values('item_id')

index2asin = {item_id:item_id_old for item_id, item_id_old in zip(sorted_ids['item_id'], sorted_ids['item_id_old'])}
asin2index = {item_id_old:item_id for item_id, item_id_old in zip(sorted_ids['item_id'], sorted_ids['item_id_old'])}

meta['description'] = meta['description'].fillna(' ') # some items may have no description 

meta['full_description'] = 'Title: ' + meta['title'] + '\n' + 'Description: ' + meta['description']

In [66]:
meta['new_id'] = meta['asin'].map(asin2index)

In [67]:
asin_unique = data['item_id_old'].unique()
meta_unique = meta[meta.asin.isin(asin_unique)].sort_values('new_id')

In [72]:
meta_unique.head(5)

,asin,description,title,imUrl,salesRank,categories,price,related,brand,full_description,new_id
705,B00004TMFE,It's in our name: Avalon Organics. Our commitm...,Avalon Biotin B-Complex Thickening Conditioner...,http://ecx.images-amazon.com/images/I/41oDCiQA...,"{'Arts, Crafts & Sewing': None, 'Automotive': ...","[[Beauty, Hair Care, Conditioners]]",9.49,"{'also_bought': ['B00JMBDCQW', 'B006VUR7N6', '...",Avalon Organics,Title: Avalon Biotin B-Complex Thickening Cond...,1.0
706,B00004TUBL,Qality designed two chamber that utilizes a 'p...,"Better Living Classic Two Chamber Dispenser, W...",http://ecx.images-amazon.com/images/I/31ERK36T...,"{'Arts, Crafts & Sewing': None, 'Automotive': ...","[[Beauty, Bath & Body, Bathing Accessories, Ba...",25.99,"{'also_bought': ['B00004TUBV', 'B000WSNM9Q', '...",Classic,Title: Better Living Classic Two Chamber Dispe...,2.0
707,B00004TUBV,"The Ulti-Mate Dispenser III, provides a perfec...",Better Living The Ulti-Mate Dispenser,http://ecx.images-amazon.com/images/I/41J763J3...,"{'Arts, Crafts & Sewing': None, 'Automotive': ...","[[Beauty, Bath & Body, Bathing Accessories, Ba...",39.95,"{'also_bought': ['B000WSNM9Q', 'B00004TUBL', '...",Better Living,Title: Better Living The Ulti-Mate Dispenser\n...,3.0
713,B00004U9UY,You've watched your favorite gardeners spend h...,Crabtree &amp; Evelyn 2792 Gardeners Hand Ther...,http://ecx.images-amazon.com/images/I/51n9hJbb...,"{'Arts, Crafts & Sewing': None, 'Automotive': ...","[[Beauty, Skin Care, Hands & Nails, Hand Cream...",18.19,"{'also_bought': ['B005E0AGX2', 'B000Z65AZE', '...",Crabtree &amp; Evelyn,Title: Crabtree &amp; Evelyn 2792 Gardeners Ha...,4.0
721,B000050B6U,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",http://ecx.images-amazon.com/images/I/31V7XJ9X...,"{'Arts, Crafts & Sewing': None, 'Automotive': ...","[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,"{'also_bought': ['B000NJE40K', 'B00005O0MZ', '...",Conair,"Title: Instant Heat Curling Iron, 1-Inch\nDesc...",5.0


## Create embeddings

In [73]:
model_ckpt = 'intfloat/e5-base-v2'

model = SentenceTransformer(model_ckpt)

item_embeddings = model.encode(
    meta_unique['full_description'].to_list(),
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=False,
)
item_embeddings.shape

Batches: 100%|██████████| 96/96 [00:17<00:00,  5.41it/s]


(6086, 768)

In [76]:
np.save('../data/embeddings/my_beauty_embs.npy', item_embeddings)